# Classificação de Mensagens SMS Usando Aprendizado de Máquina

Este estudo utiliza o conjunto de dados *SMS Spam Collection Dataset* (Kaggle) para investigar padrões textuais em mensagens SMS e aplicar técnicas de aprendizado de máquina voltadas à **detecção automática de spam**.

O dataset contém **5.572 registros** rotulados como **"ham"** (mensagem legítima) ou **"spam"** (mensagem indesejada), com a seguinte distribuição observada no arquivo: **4.825 ham** e **747 spam**.

As colunas principais incluem:

- **label** — rótulo original (`ham` ou `spam`) convertido para binário (`0` = ham, `1` = spam).  
- **message** — texto completo da mensagem SMS.  
- **message_processed** — versão pré-processada do texto (quando aplicável), com pontuação removida, stop words filtradas e lematização.

###Fonte: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset/data
---

## Objetivos

- Realizar **análise exploratória e verificação** do dataset (estrutura e balanceamento das classes).  
- Aplicar **pré-processamento de texto** (remoção de pontuação, remoção de stop words e lematização) para melhorar representações textuais.  
- Converter texto em vetores numéricos via **TF-IDF**.  
- Treinar e comparar **modelos de classificação** supervisionada:
  - **Multinomial Naive Bayes** sobre texto original e sobre texto pré-processado.  
  - **Regressão Logística** sobre texto pré-processado.  
- Avaliar desempenho com **classification report**, **matriz de confusão** e **acurácia**, e comparar métricas direcionadas à classe *spam* (Precision, Recall, F1-score).

---

## Metodologia

- **Pré-processamento:**  
  - Leitura do CSV (`encoding='latin-1'`), limpeza de colunas irrelevantes e padronização dos nomes (`label`, `message`).  
  - Conversão de rótulos para binário (`ham` → 0, `spam` → 1).  
  - Implementação opcional de pré-processamento linguístico com **NLTK**: remoção de pontuação, tokenização simples por espaços, remoção de stop words em inglês e **lematização** via `WordNetLemmatizer`.  
  - O texto processado é armazenado em `message_processed`.

- **Representação de texto:**  
  - Vetorização com **`TfidfVectorizer`** (um vetor para texto original com `stop_words='english'`, e outro para o texto processado).

- **Modelagem e avaliação:**  
  - Divisão treino/teste com `test_size=0.2` e `random_state=42`.  
  - Treinamento de **MultinomialNB** sobre texto original e sobre texto processado.  
  - Treinamento de **LogisticRegression** sobre texto processado.  
  - Avaliação com `classification_report`, `confusion_matrix` e `accuracy_score`.  
  - Construção de uma tabela comparativa com métricas de interesse, incluindo métricas focadas na classe *spam*.

---

## Importância do Estudo

A detecção de spam em SMS é uma aplicação prática clássica de **Processamento de Linguagem Natural (NLP)** e **aprendizado de máquina**, com impacto direto em redução de fraudes, phishing e do ruído nas comunicações.  

Este notebook demonstra um fluxo completo desde a leitura e pré-processamento do texto até a comparação de modelos (Naive Bayes e Regressão Logística), permitindo avaliar como transformações simples no texto (como lematização e remoção de stop words) afetam a capacidade de distinguir **spam** de **não spam**.


In [ ]:
# Importando as bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Carregando dados
df = pd.read_csv("spam.csv", encoding='latin-1')

# Visualizando as primeiras linhas
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


# Limpeza e pré processamento

In [ ]:
# Limpeza das colunas
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], errors='ignore')
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

# Conversão dos rótulos para binário
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Visualizando as primeiras linhas
print(df.head())

   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


# Análise de Spam

In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label'], test_size=0.2, random_state=42
)

# Vetorização de texto (TF-IDF)
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinamento do modelo (Naive Bayes)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [ ]:
# Avaliação do modelo
y_pred = model.predict(X_test_tfidf)

print("\n=== Relatório de Classificação ===")
print(classification_report(y_test, y_pred))

print("\n=== Matriz de Confusão ===")
print(confusion_matrix(y_test, y_pred))

print("\nAcurácia:", accuracy_score(y_test, y_pred))



=== Relatório de Classificação ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.75      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115


=== Matriz de Confusão ===
[[965   0]
 [ 37 113]]

Acurácia: 0.9668161434977578


#  Pré-processamento de Texto Aprimorado com NLTK

Primeiro, vamos instalar e importar as bibliotecas necessárias para lematização e remoção de pontuação. A lematização, diferente do stemming, tenta converter as palavras à sua forma base lexical (lema), garantindo que a palavra resultante ainda tenha significado.

In [ ]:
# Instalar NLTK (se ainda não estiver instalado)
!pip install nltk

# Importando as bibliotecas
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

# Baixar recursos necessários para NLTK
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def text_preprocess(text):
    # 1. Remover pontuação
    text = ''.join([char for char in text if char not in string.punctuation])
    # 2. Converter para minúsculas e tokenizar
    words = text.lower().split()
    # 3. Remover stop words e lematizar
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Aplicar o pré-processamento avançado às mensagens
df['message_processed'] = df['message'].apply(text_preprocess)

# Visualizar as primeiras linhas com o novo texto processado
print(df[['message', 'message_processed']].head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                             message  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                   message_processed  
0  go jurong point crazy available bugis n great ...  
1                            ok lar joking wif u oni  
2  free entry 2 wkly comp win fa cup final tkts 2...  
3                u dun say early hor u c already say  
4           nah dont think go usf life around though  


In [ ]:
# Divisão em treino e teste com o texto processado
X_train_processed, X_test_processed, y_train, y_test = train_test_split(
    df['message_processed'], df['label'], test_size=0.2, random_state=42
)

# Vetorização de texto (TF-IDF) com o texto processado
vectorizer_processed = TfidfVectorizer()
X_train_tfidf_processed = vectorizer_processed.fit_transform(X_train_processed)
X_test_tfidf_processed = vectorizer_processed.transform(X_test_processed)

# Treinamento do modelo (Naive Bayes) com o texto processado
model_processed = MultinomialNB()
model_processed.fit(X_train_tfidf_processed, y_train)

# Avaliação do modelo com texto processado
y_pred_processed = model_processed.predict(X_test_tfidf_processed)

print("\n=== Relatório de Classificação (com pré-processamento avançado) ===")
print(classification_report(y_test, y_pred_processed))
print("\nAcurácia (com pré-processamento avançado):", accuracy_score(y_test, y_pred_processed))


=== Relatório de Classificação (com pré-processamento avançado) ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.73      0.85       150

    accuracy                           0.96      1115
   macro avg       0.98      0.87      0.91      1115
weighted avg       0.97      0.96      0.96      1115


Acurácia (com pré-processamento avançado): 0.9641255605381166


# Experimentar um Modelo Diferente: Regressão Logística


In [ ]:
from sklearn.linear_model import LogisticRegression

# Treinar um modelo de Regressão Logística
logistic_model = LogisticRegression(solver='liblinear', random_state=42)
logistic_model.fit(X_train_tfidf_processed, y_train)

# Avaliação do modelo de Regressão Logística
y_pred_logistic = logistic_model.predict(X_test_tfidf_processed)

print("\n=== Relatório de Classificação (Regressão Logística) ===")
print(classification_report(y_test, y_pred_logistic))
print("\nAcurácia (Regressão Logística):", accuracy_score(y_test, y_pred_logistic))

print("\n=== Matriz de Confusão (Regressão Logística) ===")
print(confusion_matrix(y_test, y_pred_logistic))


=== Relatório de Classificação (Regressão Logística) ===
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.66      0.78       150

    accuracy                           0.95      1115
   macro avg       0.96      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115


Acurácia (Regressão Logística): 0.9506726457399103

=== Matriz de Confusão (Regressão Logística) ===
[[961   4]
 [ 51  99]]


## Extraindo Métricas dos Modelos




In [ ]:
metrics_original_nb = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision_Spam': 1.00,
    'Recall_Spam': 0.75,
    'F1-Score_Spam': 0.86
}

print("Métricas do Naive Bayes Original:", metrics_original_nb)

Métricas do Naive Bayes Original: {'Accuracy': 0.9668161434977578, 'Precision_Spam': 1.0, 'Recall_Spam': 0.75, 'F1-Score_Spam': 0.86}


In [ ]:
metrics_processed_nb = {
    'Accuracy': accuracy_score(y_test, y_pred_processed),
    'Precision_Spam': 1.00,
    'Recall_Spam': 0.73,
    'F1-Score_Spam': 0.85
}

print("Métricas do Naive Bayes com Pré-processamento:", metrics_processed_nb)

Métricas do Naive Bayes com Pré-processamento: {'Accuracy': 0.9641255605381166, 'Precision_Spam': 1.0, 'Recall_Spam': 0.73, 'F1-Score_Spam': 0.85}


In [ ]:
metrics_logistic_reg = {
    'Accuracy': accuracy_score(y_test, y_pred_logistic),
    'Precision_Spam': 0.96,
    'Recall_Spam': 0.66,
    'F1-Score_Spam': 0.78
}

print("Métricas da Regressão Logística:", metrics_logistic_reg)

Métricas da Regressão Logística: {'Accuracy': 0.9506726457399103, 'Precision_Spam': 0.96, 'Recall_Spam': 0.66, 'F1-Score_Spam': 0.78}


In [ ]:
import pandas as pd

# Criando um DataFrame para comparar as métricas
comparison_df = pd.DataFrame({
    'Modelo': ['Naive Bayes Original', 'Naive Bayes com Pré-processamento', 'Regressão Logística'],
    'Accuracy': [metrics_original_nb['Accuracy'], metrics_processed_nb['Accuracy'], metrics_logistic_reg['Accuracy']],
    'Precision_Spam': [metrics_original_nb['Precision_Spam'], metrics_processed_nb['Precision_Spam'], metrics_logistic_reg['Precision_Spam']],
    'Recall_Spam': [metrics_original_nb['Recall_Spam'], metrics_processed_nb['Recall_Spam'], metrics_logistic_reg['Recall_Spam']],
    'F1-Score_Spam': [metrics_original_nb['F1-Score_Spam'], metrics_processed_nb['F1-Score_Spam'], metrics_logistic_reg['F1-Score_Spam']]
})

print("\n=== Tabela Comparativa de Métricas (Classe SPAM) ===")
print(comparison_df.set_index('Modelo'))


=== Tabela Comparativa de Métricas (Classe SPAM) ===
                                   Accuracy  Precision_Spam  Recall_Spam  \
Modelo                                                                     
Naive Bayes Original               0.966816            1.00         0.75   
Naive Bayes com Pré-processamento  0.964126            1.00         0.73   
Regressão Logística                0.950673            0.96         0.66   

                                   F1-Score_Spam  
Modelo                                            
Naive Bayes Original                        0.86  
Naive Bayes com Pré-processamento           0.85  
Regressão Logística                         0.78  



# Principais Descobertas da Análise de Dados
*   O modelo **Naive Bayes Original** alcançou o maior F1-score para spam em 0.86, com uma precisão perfeita de 1.00 e um recall de 0.75. Sua acurácia geral foi de aproximadamente 0.967.
*   O modelo **Naive Bayes Pré-processado** apresentou um desempenho muito semelhante ao original, alcançando uma precisão perfeita de 1.00 para spam, um recall de 0.73 e um F1-score de 0.85. Sua acurácia geral foi ligeiramente inferior, de aproximadamente 0.964.
*   O modelo de **Regressão Logística** mostrou um desempenho inferior para a classe de spam em comparação com os modelos Naive Bayes, com uma precisão de 0.96, recall de 0.66 e um F1-score de 0.78. Sua acurácia geral foi a menor entre os três, de aproximadamente 0.951.
*   Ambos os modelos Naive Bayes exibiram precisão perfeita (1.00) na identificação de spam, indicando que não houve falsos positivos para a classe de spam, o que é superior à precisão de 0.96 da Regressão Logística.

### Insights:
*   **Impacto do pré-processamento:** A etapa de pré-processamento reduziu ligeiramente o desempenho do modelo Naive Bayes em termos de recall e F1-score para a classe de spam (de 0.75 para 0.73 no recall e de 0.86 para 0.85 no F1-score), sugerindo que o pré-processamento específico aplicado pode não ser benéfico ou pode ser refinado.
*   **Seleção do modelo:** Para esta tarefa de classificação de spam, os modelos Naive Bayes superam significativamente a Regressão Logística, especialmente em precisão e F1-score para a classe de spam, indicando que são mais adequados para identificar spam com precisão, minimizando falsos positivos. Investigações adicionais poderiam incluir a exploração de outras variantes de Naive Bayes ou métodos de conjunto para potencialmente aumentar o recall sem comprometer a precisão.

In [29]:
# Visualização de resultados

results_df = pd.DataFrame({'message': X_test, 'actual_label': y_test})
results_df['naive_bayes_original_prediction'] = y_pred
results_df['naive_bayes_processed_prediction'] = y_pred_processed
results_df['logistic_regression_prediction'] = y_pred_logistic

display(results_df.head(20))

,message,actual_label,naive_bayes_original_prediction,naive_bayes_processed_prediction,logistic_regression_prediction
3245,"Funny fact Nobody teaches volcanoes 2 erupt, t...",0,0,0,0
944,I sent my scores to sophas and i had to do sec...,0,0,0,0
1044,We know someone who you know that fancies you....,1,0,0,0
2484,Only if you promise your getting out as SOON a...,0,0,0,0
812,Congratulations ur awarded either å£500 of CD ...,1,1,1,1
2973,"I'll text carlos and let you know, hang on",0,0,0,0
2991,K.i did't see you.:)k:)where are you now?,0,0,0,0
2942,No message..no responce..what happend?,0,0,0,0
230,Get down in gandhipuram and walk to cross cut ...,0,0,0,0
1181,You flippin your shit yet?,0,0,0,0
